<a href="https://colab.research.google.com/github/yongjulee0213/ale_python_interface/blob/master/naturallanguage_week2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#training a binayr classifier with IMDB revies dataset

In [1]:
#check the version of tensorflow
import tensorflow as tf
print(tf.__version__)

2.8.2


In [2]:
#tensorflow dataset
import tensorflow_datasets as tfds
imdb, info=tfds.load("imdb_reviews",with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVQEZIV/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVQEZIV/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVQEZIV/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [15]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        '

In [16]:
print(imdb)#어떻게 나눌것인지 label확인

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [18]:
#예시 문장 2개 확인
for example in imdb['train'].take(2):
  print(example)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [3]:
#split test & training dataset
import numpy as np
#각 문장은 25,000개의 문장과 label을 텐서로 포함.
train_data, test_data=imdb['train'],imdb['test']

training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [4]:
#postivie : 1, negative:0


In [5]:
#마지막으로 나오는 레이블은 numpy array임
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [6]:
#문장 토큰화
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok="<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences, maxlen=max_length)

In [7]:
#뉴런 네트워크 구성
#the results of the embedding  : 2d array with the length of the sentence
#embeding dimension :size
# 따라서, 우리는 flatten/globalaveragePooling1d을 통해 이미지를 평평하게 만들어야함
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [10]:
model.fit(padded,training_labels_final,epochs=10,validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 0.6013 - acc: 0.7444 - val_loss: 0.4620 - val_acc: 0.8207
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3919 - acc: 0.8376 - val_loss: 0.3462 - val_acc: 0.8519
Epoch 3/10
782/782 [==============================] - 4s 6ms/step - loss: 0.3230 - acc: 0.8632 - val_loss: 0.3243 - val_acc: 0.8610
Epoch 4/10
782/782 [==============================] - 4s 6ms/step - loss: 0.2932 - acc: 0.8776 - val_loss: 0.3237 - val_acc: 0.8595
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2750 - acc: 0.8873 - val_loss: 0.3352 - val_acc: 0.8554
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2625 - acc: 0.8921 - val_loss: 0.3287 - val_acc: 0.8580
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2521 - acc: 0.8980 - val_loss: 0.3352 - val_acc: 0.8556
Epoch 8/10
782/782 [==============================] - 8s 10ms/step - loss: 0

In [20]:
#visualize word embeddings
#get the embedding first layer from the model
embedding_layer=model.layers[0]

#get the weight of the embedding layer
embedding_weights=embedding_layer.get_weights()[0]

#print the shape (vocab_size단어가 몇개 있고 , embedding_dim레이어로 들어갈때 차원수)
print(embedding_weights.shape)


(10000, 16)


In [23]:
#need to generate two files
'''
vecs.tsv : contains the vector weights of each word in the vocab
meta.tsv : contains the words in the vocab
'''

'\nvecs.tsv : contains the vector weights of each word in the vocab\nmeta.tsv : contains the words in the vocab\n'

In [24]:
#to be able to plot is, we need to reverse our word index
reverse_word_index=tokenizer.index_word #인덱스 주기 위해서 리버스

In [28]:
#write the vectors and metadata auto files
import io
out_v=io.open('vecs.tsv','w',encoding='utf-8')
out_m=io.open('meta.tsv','w',encoding='utf-8')

for word_num in range(1,vocab_size):
  #get the word associataed at the current index
  word_name=reverse_word_index[word_num]
  
  #get the embedding weigths associated at the current index
  word_embeddings=embedding_weights[word_num]

  #write the word name
  out_m.write(word_name+"\n")#메타데이터에는 그냥 단어를 적는다

  #write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embeddings])+"\n")#벡터 파일에 단순히 embedding array에 있는 각 항목의 값, 즉 단어에 대한 벡터의 각 차원 계수를 씀

#close the files
out_v.close()
out_m.close()

In [30]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>